In [1]:
import os
import csv
import sys
import scipy.optimize as opt
import scipy.stats as stat
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la

pi = np.pi
sin = np.sin
cos = np.cos

In [28]:
def fillin2(data):
    """
    Fills in blanks of arrays without shifting frames by the starting frame.  Compare to fillin.

    Input: trajectory dataset from MOSAIC tracking software read into a numpy array
    Output: modified numpy array with missing frames filled in.
    """

    shap = int(max(data[:, 1])) + 1
    shape1 = int(min(data[:, 1]))
    newshap = shap - shape1
    filledin = np.zeros((newshap, 5))
    filledin[0, :] = data[0, :]

    count = 0
    new = 0
    other = 0
    tot = 0

    for num in range(1, newshap):
        if data[num-new, 1]-data[num-new-1, 1] == 1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new, 1]-data[num - new - 1, 1] > 1:
            new = new + 1
            iba = int(data[num - new+1, 1]-data[num - new, 1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            # dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)

        else:
            other = other + 1
        tot = count + new + other

    return filledin


def MSD_iteration(folder, name, cut, totvids, conversion, frames):
    """
    Cleans up data for MSD analysis from csv files.  Outputs in form of
    dictionaries.
    """

    trajectory = dict()
    tots = dict()  # Total particles in each video
    newtots = dict()  # Cumulative total particles.
    newtots[0] = 0
    tlen = dict()
    tlength = dict()
    tlength[0] = 0

    for num in range(1, totvids + 1):
        trajectory[num] = np.genfromtxt(folder+'Traj_{}_{}.tif.csv'.format(name, num), delimiter=",")
        trajectory[num] = np.delete(trajectory[num], 0, 1)

        tots[num] = trajectory[num][-1, 0].astype(np.int64)
        newtots[num] = newtots[num-1] + tots[num]

        tlen[num] = trajectory[num].shape[0]
        tlength[num] = tlength[num-1] + tlen[num]

    placeholder = np.zeros((tlength[totvids], 11))

    for num in range(1, totvids + 1):
        placeholder[tlength[num-1]:tlength[num], :] = trajectory[num]
        placeholder[tlength[num-1]:tlength[num], 0] = placeholder[tlength[num-1]:tlength[num], 0] + newtots[num-1]

    dataset = dict()
    rawdataset = np.zeros(placeholder.shape)
    particles = placeholder[:, 0]
    total = int(max(particles))
    total1 = total + 1
    rawdataset = placeholder[:, :]

    fixed = np.zeros(placeholder.shape)
    fixed[:, 0:2] = rawdataset[:, 0:2]
    fixed[:, 2:4] = conversion[0] * rawdataset[:, 2:4]
    fixed[:, 4] = conversion[2] * rawdataset[:, 4]

    x = np.zeros((frames, total1 - 1))
    y = np.zeros((frames, total1 - 1))
    xs = np.zeros((frames, total1 - 1))
    ys = np.zeros((frames, total1 - 1))

    nones = 0
    cutoff = cut
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)

        if max1 - min1 < cutoff:
            nones = nones + 1
        else:
            holdplease = fillin2(fixed[min1:max1+1, 0:5])
            x[int(holdplease[0, 1]):int(holdplease[-1, 1])+1, num - nones - 1] = holdplease[:, 2]
            y[int(holdplease[0, 1]):int(holdplease[-1, 1])+1, num - nones - 1] = holdplease[:, 3]

            xs[0:int(holdplease[-1, 1])+1-int(holdplease[0, 1]), num - nones - 1] = holdplease[:, 2]
            ys[0:int(holdplease[-1, 1])+1-int(holdplease[0, 1]), num - nones - 1] = holdplease[:, 3]

    total1 = total1 - nones - 1
    x_m = x[:, :total1-1]
    y_m = y[:, :total1-1]
    xs_m = xs[:, :total1-1]
    ys_m = ys[:, :total1-1]

    return total1, xs_m, ys_m, x_m, y_m


def vectorized_MMSD_calcs(frames, total1, xs_m, ys_m, x_m, y_m, frame_m):

    SM1x = np.zeros((frames, total1-1))
    SM1y = np.zeros((frames, total1-1))
    SM2xy = np.zeros((frames, total1-1))

    xs_m = ma.masked_equal(xs_m, 0)
    ys_m = ma.masked_equal(ys_m, 0)

    x_m = ma.masked_equal(x_m, 0)
    y_m = ma.masked_equal(y_m, 0)

    geoM1x = np.zeros(frame_m)
    geoM1y = np.zeros(frame_m)

    for frame in range(1, frame_m):
        bx = xs_m[frame, :]
        cx = xs_m[:-frame, :]
        Mx = (bx - cx)**2

        Mxa = np.mean(Mx, axis=0)
        # Mxab = np.mean(np.log(Mxa), axis=0)

        # geoM1x[frame] = Mxab

        by = ys_m[frame, :]
        cy = ys_m[:-frame, :]
        My = (by - cy)**2

        Mya = np.mean(My, axis=0)
        # Myab = np.mean(np.log(Mya), axis=0)

        # geoM1y[frame] = Myab
        SM1x[frame, :] = Mxa
        SM1y[frame, :] = Mya

    geoM2xy = np.mean(ma.log(SM1x+SM1y), axis=1)
    gSEM = stat.sem(ma.log(SM1x+SM1y), axis=1)
    SM2xy = SM1x + SM1y

    return geoM2xy, gSEM, SM1x, SM1y, SM2xy

In [29]:
folder = "./"
path = "./geoM2xy_{sample_name}.csv"
frames = 651
SD_frames = [10, 20, 50, 80]
conversion = (0.16, 100.02, 1)  # (0.3, 3.95, 1)
to_frame = 120
dimension = "2D"
time_to_calculate = 1

base = "in_agarose"
base_name = "RED"
test_bins = np.linspace(0, 75, 76)

# name = 'RED_KO_PEG_P1_S1_cortex'
cut = 1
totvids = 2
frame_m = 651  # atm I can't go lower than the actual value.

parameters = {}
parameters["channels"] = ["RED"]
parameters["surface functionalities"] = ["nPEG"]
parameters["slices"] = ["S1", "S2"]
parameters["videos"] = [1, 2]
parameters["replicates"] = [1, 2]


channels = parameters["channels"]
surface_functionalities = parameters["surface functionalities"]
slices = parameters["slices"]
videos = parameters["videos"]
replicates = parameters["replicates"]

geoM2xy = {}
gSEM = {}
SM1x = {}
SM1y = {}
SM2xy = {}

In [30]:
for channel in channels:
    for surface_functionality in surface_functionalities:
        slice_counter = 0
        for slic in slices:
            for video in videos:
                sample_name = "{}_{}_{}_{}_{}".format(channel, surface_functionality, '37C_pH72', slic, video)
                DIR = folder

                total1, xs, ys, x, y = MSD_iteration(DIR, sample_name, cut, totvids, conversion, frame_m)

                geoM2xy[sample_name], gSEM[sample_name], SM1x[sample_name], SM1y[sample_name],\
                    SM2xy[sample_name] = vectorized_MMSD_calcs(frames, total1, xs, ys, x, y, frame_m)
                np.savetxt(DIR+'geoM2xy_{}.csv'.format(sample_name), geoM2xy[sample_name], delimiter=',')
                np.savetxt(DIR+'gSEM_{}.csv'.format(sample_name), gSEM[sample_name], delimiter=',')
                np.savetxt(DIR+'SM2xy_{}.csv'.format(sample_name), SM2xy[sample_name], delimiter=',')

                slice_counter = slice_counter + 1

In [12]:
SM2xy1 = SM2xy['RED_nPEG_37C_pH72_S1_1']

In [18]:
geoM2xy1 = np.mean(ma.log(SM2xy1+SM2xy1), axis=1)

In [20]:
geoM2xy1 = stat.sem(ma.log(SM2xy1+SM2xy1), axis=1)

In [21]:
geoM2xy

array([ 0.        ,  0.08080702,  0.10318802,  0.09891259,  0.09218002,
        0.08695801,  0.08191538,  0.07787173,  0.07386894,  0.07153525,
        0.06986269,  0.0677853 ,  0.06817925,  0.06713191,  0.06544003,
        0.06434326,  0.06463163,  0.06243887,  0.06439937,  0.06076896,
        0.06281202,  0.06024286,  0.06101672,  0.05546603,  0.05897295,
        0.05390863,  0.0592696 ,  0.05865116,  0.05621035,  0.05509468,
        0.05768897,  0.05565244,  0.05728605,  0.05587126,  0.05649437,
        0.05265252,  0.0555687 ,  0.05354089,  0.05134968,  0.05016551,
        0.0529781 ,  0.04995147,  0.05317292,  0.05213238,  0.05200264,
        0.05096998,  0.05074257,  0.05103048,  0.05128592,  0.04962384,
        0.05090041,  0.04865657,  0.05051277,  0.05031416,  0.05088686,
        0.05134718,  0.0480223 ,  0.04939072,  0.04996471,  0.05048015,
        0.04950887,  0.05086384,  0.04957504,  0.05078168,  0.04972348,
        0.04940347,  0.04997614,  0.04872188,  0.05078497,  0.05

In [31]:
geoM2xy[sample_name]

masked_array(data = [-- -5.528753793045865 -5.154351076507821 -4.9884488061401955
 -4.8704793432450515 -4.789961056015163 -4.856133493288042
 -4.795382746192625 -4.980337433401773 -5.054798104552319 -5.03000562725889
 -5.107536635763646 -5.291879340082459 -5.3797722708582425
 -5.1928142072571974 -5.3788559610864 -5.510558589644675 -5.354729399502736
 -5.644407890771462 -5.558580574713081 -5.563326025821122
 -5.376008304871114 -5.584534543815228 -5.669268052259213
 -5.6915602182245655 -5.633394523530452 -5.398013577246282
 -5.688157653557689 -5.53202189922901 -5.766638574097802 -5.872013863177209
 -6.000530815633388 -5.851926719943082 -5.753195017050547
 -5.421343407949139 -5.974886120292065 -5.929729757954048
 -5.975597912778438 -5.706651999981877 -6.0109711491532885
 -5.844429679295701 -5.836705125965918 -5.787688462153361
 -5.803726430930447 -5.964526854420684 -5.849810071350376
 -5.890529671077279 -5.926957863927074 -5.779820267830473
 -5.985928239804955 -5.7318265653385385 -5.90533